# Demonstration of pyQ3 python wrapper for DEW-ENKI-EQ3 aqueous speciation calculations
First import the required packages:

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import DEW_tools as dew
from thermoengine import model, phases, chem
%run core.ipynb
import dill

Import the coder objects representing the aqueous species (created using the import_DEW.ipynb notebook)

In [16]:
load_coder_modules(working_dir='dew2019_coderfiles')
with open('DEW2019.pkl','rb') as file:
    dewdb = dill.load(file)
    file.close()
print('Number of species = '+str(len(dewdb)))

Number of species = 228


## TO DO LIST:
- Check that the default species align with all DATA0 files- i.e. that they don't change with pressure and temperature.

In [17]:
chem.get_Berman_formula(dewdb['CO2(AQ)'].props['element_comp'][0])

'C(1.0)O(2.0)'

In [22]:
[1,2,3,4][:-1]

## Create a custom DATA0 file
One use for the code is to automatically create a customised DATA0 file for use with EQPT and EQ3 externally. This can use the full set of aqueous species, or a custom system can be specified.

In [ ]:
chem.get_Berman_formula

In [18]:
berman = model.Database()

In [20]:
h2o = berman.get_phase('H2O')

In [21]:
h2o.abbrev

'H2O'

## Calculate speciation for a fluid with defined molalities
The code can be used to calculate the speciation of a solution which is not in equilibrium with minerals

## Calculate fluid composition and speciation for a fluid in equilibrium with a rock